In [1]:
from PIL import Image
import numpy as np
import os
import torch
from diffusers import DDIMScheduler,StableDiffusionXLInpaintPipeline
from torchvision.utils import save_image
from pytorch_lightning import seed_everything
from matplotlib import pyplot as plt
torch.cuda.set_device(0)  # set the GPU device

/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-26 07:11:24.638457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-26 07:11:25.684984: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
#scheduler = DPMSolverMultistepScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear",solver_order=3)
base = StableDiffusionXLInpaintPipeline.from_pretrained(
    "/hy-tmp/stable-diffusion-xl-base-1.0",
    #custom_pipeline="/hy-tmp/MyREMOVAL/SDXL_inp_pipeline.py",
    scheduler=scheduler,
    variant="fp16",
    use_safetensors=True,
    torch_dtype=torch.float16,
).to(device)

""" refiner = StableDiffusionXLInpaintPipeline.from_pretrained(
    "/hy-tmp/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    scheduler=scheduler,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to(device) """

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  3.95it/s]


' refiner = StableDiffusionXLInpaintPipeline.from_pretrained(\n    "/hy-tmp/stable-diffusion-xl-refiner-1.0",\n    text_encoder_2=base.text_encoder_2,\n    scheduler=scheduler,\n    vae=base.vae,\n    torch_dtype=torch.float16,\n    use_safetensors=True,\n    variant="fp16",\n).to(device) '

In [3]:
image_path = "/hy-tmp/DATA/sample/62c75e993beb5e37_m02p0tk3_8e0102e2.jpg"
mask_image_path = "/hy-tmp/DATA/sample/62c75e993beb5e37_m02p0tk3_8e0102e2_mask.png"
prompt = ""
image_sc = Image.open(image_path).convert('RGB')
mask = Image.open(mask_image_path)
seed=777
g = torch.Generator('cuda').manual_seed(seed)

In [4]:
image_base = base(
    prompt=prompt,
    image=image_sc,
    height=1024,
    width=1024,
    strength=0.95,
    mask_image=mask,
    generator=g,
    num_inference_steps=50,
    guidance_scale=1,
).images[0]
image_base.save("result_+inp.png") 

100%|██████████| 47/47 [00:06<00:00,  6.90it/s]


In [5]:
""" #image = refiner(prompt=prompt, mask_image = mask,generator=g,image=image_base[None, :],num_inference_steps=50,strength=0.3).images[0]
image = refiner(prompt=prompt, mask_image = mask,generator=g,image=image_sc,num_inference_steps=50,strength=0.3).images[0]
image.save("result_+ref.png")
image """

' #image = refiner(prompt=prompt, mask_image = mask,generator=g,image=image_base[None, :],num_inference_steps=50,strength=0.3).images[0]\nimage = refiner(prompt=prompt, mask_image = mask,generator=g,image=image_sc,num_inference_steps=50,strength=0.3).images[0]\nimage.save("result_+ref.png")\nimage '